In [1]:
import pandas as pd
import numpy as np
import math
import glob

In [2]:
def getListBack(df, item, trial):
    trial = df[item][trial]
    trial = trial.replace('[', '')
    trial = trial.replace(']', '')
    trial = trial.split(',')
    trial = [float(i) for i in trial]
    return trial

In [3]:
def windowMSE2(df, participant, frameRate, all_mouse_x, all_mouse_y, all_target_x, all_target_y, windowlist):
    print('Starting MSE calculation...')
    for window in windowlist:
        standard_error = [] #Make list with standard error
        for i in range(0,len(all_mouse_x)):
            if type(all_target_y[i]) == str:
                standard_error.append(all_target_y[i])
            else:
                p = [all_mouse_x[i], all_mouse_y[i]]
                q = [all_target_x[i], all_target_y[i]]
                standard_error.append(math.dist(p, q))

        all_error = 0
        for i, e in enumerate(standard_error):
            if type(e) == str:
                loop = 1
                count = 0
                while count != int(window*frameRate): #30 sec
                    cur = standard_error[i-int(3*frameRate)-loop]
                    if type(cur) == str:
                        loop += 1
                    else:
                        all_error += cur
                        loop += 1
                        count += 1 
                result2.loc[len(result2)] = {'Participant': participant, 'Group': df['group'][0], 'window': window, 'report': 'self', 'MW': 'a', 'MSD': all_error/int(window*frameRate)}
                all_error = 0
                
                loop = 1
                count = 0
                while count != int(window*frameRate): #30 sec
                    cur = standard_error[i+int(1*frameRate)+loop]
                    if type(cur) == str:
                        loop += 1
                    else:
                        all_error += cur
                        loop += 1
                        count += 1 
                result2.loc[len(result2)] = {'Participant': participant, 'Group': df['group'][0], 'window': window, 'report': 'self', 'MW': 'f', 'MSD': all_error/int(window*frameRate)}
                all_error = 0

def selfCaught(current_data, participant):
    df = current_data
    frameRate = current_data['frameRate'][0]
    
    all_mouse_x = [0.0] * int(frameRate*35) #35 = 32s trial + 3s delay
    all_mouse_y = [0.0] * int(frameRate*35)
    all_target_x = [0.0] * int(frameRate*35)
    all_target_y = [0.0] * int(frameRate*35)
    
    print('Loading trials...')    
    for cur_trial in range (1,55): #Loop over trials
        if df['mouse.x'][cur_trial] == '':
            continue        
        
        mouse_x = getListBack(df, 'mouse.x', cur_trial) #Get mouse x, y  positions and t
        mouse_y = getListBack(df, 'mouse.y', cur_trial)
        mouse_t = getListBack(df, 'mouse.time', cur_trial)
        
        if df['selfReport.rt'][cur_trial] == '' and len(mouse_x) > 32: #Get current frameRate (this can change)
            frameRate = len(mouse_x)/32  

        t = [] #Get time
        for time in range(0, int(frameRate*32)):
            t.append(time/int(frameRate))
        
        if len(t) > len(mouse_x):                
            frameRate=(len(mouse_x)-120)/32
            t = [] #Get time
            for time in range(0, int(frameRate*32)):
                t.append(time/int(frameRate))
        
        if df['selfReport.rt'][cur_trial] != '': #Is there a self report in this trial?
            print("Trial {}: mouse(x,y,t) = ({},{},{})".format(cur_trial, len(mouse_x), len(mouse_y), len(t)))
            
            self_times = getListBack(df, 'selfReport.rt', cur_trial) #Get self reports time
            dif = int(round((len(mouse_x)-(int(frameRate*32)))/len(self_times),0)) #length of answer
            
            for idx, self_report in enumerate(self_times):
                self_index = 0
                for index, time in enumerate(mouse_t): #Find index of report
                    if time == 'space':
                        continue
                    elif time > self_report and self_index == 0:
                        self_index = index
                
                if self_report == self_times[-1]:
                    if self_index > len(t):
                        while len(mouse_x) != len(t):
                            mouse_x.pop(-1)
                            mouse_y.pop(-1)
                            mouse_t.pop(-1)
                        mouse_x.append('space')
                        mouse_y.append('space')
                        mouse_t.append('space')
                        t.append('space') 
                    else:
                        mouse_x.insert(self_index, 'space')
                        mouse_y.insert(self_index, 'space')
                        mouse_t.insert(self_index, 'space')
                        t.insert(self_index, 'space') 

                        while len(mouse_x) != len(t):
                            mouse_x.pop(self_index+1)
                            mouse_y.pop(self_index+1)
                            mouse_t.pop(self_index+1)
                
                elif self_report > 2 and self_index-int(frameRate*2) > 0:
                    mouse_x.insert(self_index, 'space')
                    mouse_y.insert(self_index, 'space')
                    mouse_t.insert(self_index, 'space')
                    t.insert(self_index, 'space')
                    
                    del mouse_x[self_index+1:self_index+dif+1] #Remove thinking time after self
                    del mouse_y[self_index+1:self_index+dif+1]
                    del mouse_t[self_index+1:self_index+dif+1]
                    
                else: #Self in first 2 seconds
                    del mouse_x[self_index+1:self_index+dif+1] #Remove thinking time after self
                    del mouse_y[self_index+1:self_index+dif+1]
                    del mouse_t[self_index+1:self_index+dif+1]
                    
                    mouse_x.insert(0, 'space') #Insert self
                    mouse_y.insert(0, 'space')
                    mouse_t.insert(0, 'space')
                    t.insert(0, 'space') 

            current_space = mouse_x.count('space')
            mouse_x = mouse_x[int(-1*frameRate*30)-current_space:] #Remove first 2 seconds of trial
            mouse_y = mouse_y[int(-1*frameRate*30)-current_space:]
            mouse_t = mouse_t[int(-1*frameRate*30)-current_space:]
            t = t[int(-1*frameRate*30)-current_space:]
            
            if mouse_x.count('space') < current_space:
                mouse_x.insert(0, 'space') #Insert self
                mouse_y.insert(0, 'space')
                mouse_t.insert(0, 'space')
                t.insert(0, 'space')                 
                
        else: #Only remove first 2 seconds from trail
            mouse_x = mouse_x[int(-1*frameRate*30):]
            mouse_y = mouse_y[int(-1*frameRate*30):]
            mouse_t = mouse_t[int(-1*frameRate*30):]
            t = t[int(-1*frameRate*30):]          
                 
        target_x = [] #Get target positions
        target_y = []
        for n, pt in enumerate(t):
            if type(pt) == str:
                target_x.append(pt)
                target_y.append(pt)
            else:
                target_x.append(-0.8+(pt-2)/18.6)
                target_y.append((df['ab0'][cur_trial]+
                                 df['a1'][cur_trial]*math.sin(1*pt/5)+df['b1'][cur_trial]*math.cos(1*pt/5)+
                                 df['a2'][cur_trial]*math.sin(2*pt/5)+df['b2'][cur_trial]*math.cos(2*pt/5)+
                                 df['a3'][cur_trial]*math.sin(3*pt/5)+df['b3'][cur_trial]*math.cos(3*pt/5)+
                                 df['a4'][cur_trial]*math.sin(4*pt/5)+df['b4'][cur_trial]*math.cos(4*pt/5)+
                                 df['a5'][cur_trial]*math.sin(5*pt/5)+df['b5'][cur_trial]*math.cos(5*pt/5)+
                                 df['a6'][cur_trial]*math.sin(6*pt/5)+df['b6'][cur_trial]*math.cos(6*pt/5))/50)

        all_mouse_x = all_mouse_x + mouse_x #Make long list with all trials
        all_mouse_y = all_mouse_y + mouse_y
        all_target_x = all_target_x + target_x
        all_target_y = all_target_y + target_y
        
    all_mouse_x += [0.0] * int(frameRate*35) #35 = 32s trial + 3s delay
    all_mouse_y += [0.0] * int(frameRate*35)
    all_target_x += [0.0] * int(frameRate*35)
    all_target_y += [0.0] * int(frameRate*35)    

    windowMSE2(df, participant, frameRate, all_mouse_x, all_mouse_y, all_target_x, all_target_y, [5,10,15,20,25,30])

In [4]:
result2 = pd.DataFrame(columns = ['Participant', 'Group', 'window', 'report', 'MW', 'MSD'])

In [5]:
for file_name in glob.glob(r"C:\Users\Robyn\Downloads\!experiment\data\SONA"+'\*.csv'):
    participant_nr = file_name[47:52]
    print('Participant: ', participant_nr)
    data = pd.read_csv('{}'.format(file_name))
    data = data.replace(np.nan, '')
    if (data['selfReport.keys'].values == '').sum() == len(data):
        print('No self report')
    else:
        selfCaught(data, participant_nr)
    print('Done. Current results =', len(result2), '\n')

Participant:  11003
Loading trials...
Trial 27: mouse(x,y,t) = (2231,2231,1920)
Trial 28: mouse(x,y,t) = (2520,2520,1920)
Trial 29: mouse(x,y,t) = (1992,1992,1920)
Trial 30: mouse(x,y,t) = (2212,2212,1920)
Trial 31: mouse(x,y,t) = (2090,2090,1920)
Trial 32: mouse(x,y,t) = (2127,2127,1920)
Trial 33: mouse(x,y,t) = (2088,2088,1920)
Trial 34: mouse(x,y,t) = (2128,2128,1920)
Trial 35: mouse(x,y,t) = (2608,2608,1920)
Trial 36: mouse(x,y,t) = (2384,2384,1920)
Trial 37: mouse(x,y,t) = (2029,2029,1920)
Trial 38: mouse(x,y,t) = (2159,2159,1920)
Trial 39: mouse(x,y,t) = (2086,2086,1920)
Trial 40: mouse(x,y,t) = (2013,2013,1920)
Trial 41: mouse(x,y,t) = (2009,2009,1920)
Trial 42: mouse(x,y,t) = (2318,2318,1920)
Trial 43: mouse(x,y,t) = (2732,2732,1920)
Trial 44: mouse(x,y,t) = (1981,1981,1920)
Trial 45: mouse(x,y,t) = (2186,2186,1920)
Trial 47: mouse(x,y,t) = (2026,2026,1921)
Trial 48: mouse(x,y,t) = (1984,1984,1921)
Trial 49: mouse(x,y,t) = (2179,2179,1921)
Trial 51: mouse(x,y,t) = (1987,1987,19

Done. Current results = 2544 

Participant:  44055
Loading trials...
Trial 28: mouse(x,y,t) = (2037,2037,1921)
Trial 32: mouse(x,y,t) = (1975,1975,1919)
Trial 35: mouse(x,y,t) = (1982,1982,1920)
Trial 37: mouse(x,y,t) = (1943,1943,1920)
Trial 42: mouse(x,y,t) = (1941,1941,1920)
Trial 44: mouse(x,y,t) = (1929,1929,1920)
Starting MSE calculation...
Done. Current results = 2616 

Participant:  49612
Loading trials...
Trial 2: mouse(x,y,t) = (8654,8654,7645)
Trial 3: mouse(x,y,t) = (10823,10823,7645)
Trial 4: mouse(x,y,t) = (21800,21800,7645)
Trial 6: mouse(x,y,t) = (9945,9945,7653)
Trial 7: mouse(x,y,t) = (9667,9667,7653)
Trial 10: mouse(x,y,t) = (32522,32522,7667)
Trial 13: mouse(x,y,t) = (14524,14524,7654)
Trial 14: mouse(x,y,t) = (8059,8059,7654)
Trial 18: mouse(x,y,t) = (14253,14253,7645)
Trial 21: mouse(x,y,t) = (10356,10356,7636)
Trial 25: mouse(x,y,t) = (10595,10595,7636)
Trial 26: mouse(x,y,t) = (8611,8611,7636)
Starting MSE calculation...
Done. Current results = 2760 

Participan

Starting MSE calculation...
Done. Current results = 4812 

Participant:  86901
Loading trials...
Trial 28: mouse(x,y,t) = (2290,2290,1921)
Trial 33: mouse(x,y,t) = (2924,2924,1920)
Trial 36: mouse(x,y,t) = (2450,2450,1921)
Trial 45: mouse(x,y,t) = (2432,2432,1920)
Trial 47: mouse(x,y,t) = (2531,2531,1921)
Trial 52: mouse(x,y,t) = (2424,2424,1920)
Starting MSE calculation...
Done. Current results = 4908 

Participant:  87869
Loading trials...
Trial 43: mouse(x,y,t) = (10792,10792,1921)
Trial 50: mouse(x,y,t) = (6360,6360,1921)
Starting MSE calculation...
Done. Current results = 4968 

Participant:  91119
Loading trials...
Trial 11: mouse(x,y,t) = (6803,6803,1917)
Trial 15: mouse(x,y,t) = (2806,2806,1916)
Trial 25: mouse(x,y,t) = (2592,2592,1920)
Trial 26: mouse(x,y,t) = (2375,2375,1920)
Starting MSE calculation...
Done. Current results = 5016 

Participant:  92839
Loading trials...
Trial 8: mouse(x,y,t) = (2713,2713,1918)
Trial 19: mouse(x,y,t) = (3029,3029,1919)
Starting MSE calculatio

In [6]:
result2.to_csv('self_results.csv')